# How to query JSON-LD files with RDF/SPARQL , and raw JSON.
A notebook on reading in all the experiments and extracting all files with `-pending` in the name, and the relevant keys that still need to be filled in `pending`. 

In [65]:
from cmipld import get, resolve_prefix
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.plugins.sparql import prepareQuery

## Using RDFlib
Another way to load this information is using the RDF library. This allows us to apply SPARQL queries on our JSONLD files. 

In [67]:
# start by resolving the URL (we have not monkeypatched RDFLib to use cmipld prefixes yet)
url = resolve_prefix('cmip7:experiment/graph.json' )

Substituting prefix:
cmip7: https://wcrp-cmip.github.io/CMIP7-CVs/experiment/graph.json


#### Load our experiments 'graph' file into the RDF graph. 
We can also load individual file this way, but a single grouped file is easier. 

In [ ]:
g = Graph()
g.parse(location = url, format="json-ld")  # or use data=json_string

# # To preview the JSON contents we can use:
# print(g.serialize(format='json-ld'))

<Graph identifier=Nc5e9a9bb8ed9467ca7c4cffa55800d69 (<class 'rdflib.graph.Graph'>)>

In [76]:

# --- SPARQL query ---
query = """

# Select subjects ending with "-pending" or predicates containing "pending"
# and return grouped, shortened predicates and objects

SELECT ?subject
       (GROUP_CONCAT(DISTINCT ?p_short; separator=", ") AS ?predicates)
       (GROUP_CONCAT(DISTINCT ?o_short; separator=", ") AS ?objects)
WHERE {
  ?s ?p ?o .

  # Keep only subjects ending in '-pending' or predicates containing 'pending' (case-insensitive)
  FILTER (regex(str(?s), "-pending$") || regex(str(?p), "pending", "i"))

  # Shorten the subject for readability
  # Example: "https://wcrp-cmip.github.io/CMIP7-CVs/experiment/tipmip-pending"
  # becomes "tipmip-pending.json"
  BIND(CONCAT(STRAFTER(STR(?s), "experiment/"), ".json") AS ?subject)

  # Shorten predicate to just the last part after 'Experiment.'
  # Example: "...Experiment.model_realms" -> "model_realms"
  BIND(STRAFTER(STR(?p), "Experiment.") AS ?p_short)

  # Shorten object if it’s a CMIP7 experiment URI
  # Example: ".../experiment/tipmip" -> "tipmip"
  BIND(STRAFTER(STR(?o), "https://wcrp-cmip.github.io/CMIP7-CVs/experiment/") AS ?o_short)
}
GROUP BY ?subject
ORDER BY ?subject
"""




In [77]:

# --- Run query ---
results = g.query(query)

# --- Print grouped results ---
for row in results:
    print(f"Subject: {row.subject}")
    print(f"  Predicates: {row.predicates}")
    print(f"  Objects: {row.objects}")
    print()

Subject: amip-irr-pending.json
  Predicates: activity, description, tier, parent_experiment, ui_label, , start_date, model_realms, validation_key, minimum_number_of_years
  Objects: irrmip, , pending

Subject: amip-noirr-pending.json
  Predicates: model_realms, , start_date, minimum_number_of_years, description, activity, tier, parent_experiment, validation_key, ui_label
  Objects: pending, , irrmip

Subject: dcppa-assim-pending.json
  Predicates: ui_label, , model_realms, minimum_number_of_years, start_date, description, tier, activity, parent_experiment, validation_key
  Objects: , pending, dcpp

Subject: dcppa-hindcast-pending.json
  Predicates: validation_key, description, tier, activity, parent_experiment, ui_label, , model_realms, minimum_number_of_years, start_date
  Objects: , dcpp, pending

Subject: dcppb-forecast-pending.json
  Predicates: description, tier, activity, parent_experiment, ui_label, , minimum_number_of_years, start_date, model_realms, validation_key
  Objects: ,

## Using pandas and the raw json file

In [ ]:
import pandas as pd

In [96]:
json_data = get('cmip7:experiment/graph.json', depth=0)

In [ ]:
df = pd.DataFrame(json_data['@graph']).set_index('@id').sort_values(by='@type').drop(columns=['@type','model_realms']).replace('pending', '').fillna('')

mask = df.index.astype(str).str.contains(r'-pending(?:\.json)?$')
df = df[mask].set_index('validation_key')

for col in ['required', 'optional']:
    df[f"{col}_model_realms"] = ""

df

,activity,alias,description,minimum_number_of_years,parent_experiment,start_date,tier,ui_label,end,min_number_yrs_per_sim,start,branch_date,required_model_realms,optional_model_realms
validation_key,,,,,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl4p0,tipmip,[],,300,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl4p0-50y-dn2p0,tipmip,[],,200,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl4p0-50y-dn2p0-gwl2p0,tipmip,[],,300,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl3p0,tipmip,[],,300,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl2p0,tipmip,[],,300,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl2p0-50y-dn2p0,tipmip,[],,100,,,,,,,,,,
tipmip-provisional-esm-up2p0-gwl1p5,tipmip,[],,300,,,,,,,,,,
scen7-vl,fast-track,[],,75,,,,,,,,,,
scen7-vl-ext,ismip7,[],,50,,,,,,,,,,


#### We now want to write the data to a spreadsheet. We can use openpyxl for this purpose. 

In [115]:
df_scen7 = df[df.index.str.contains('scen7')]
df_tipmip = df[df.index.str.contains('tipmip')]
df_neither = df[~(df.index.str.contains('scen7') | df.index.str.contains('tipmip'))]

In [117]:
with pd.ExcelWriter('pending-experiments.xlsx', mode='w', engine='openpyxl') as writer:
    
    df_scen7.to_excel(writer, sheet_name='scen7')
    df_tipmip.to_excel(writer, sheet_name='tipmip')
    df_neither.to_excel(writer, sheet_name='other')